In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [4]:
df = pd.read_csv('df.txt', sep=' ')
test = pd.read_csv('test.txt', sep=' ')
df = df.drop_duplicates(['instance_id'])

df.shape
df.head()
test.shape
test.head()

In [3]:
test['is_trade'] = -1

df = pd.concat([df, test])

In [18]:
#time
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt


def convert_data(data):
    data['time'] = data.context_timestamp.apply(timestamp_datetime)
    data['day'] = data.time.apply(lambda x: int(x[8:10]))
    data['hour'] = data.time.apply(lambda x: int(x[11:13]))
    data['minute'] = data.time.apply(lambda x: int(x[14:15])) * 10
    user_query_day = data.groupby(
        ['user_id',
         'day']).size().reset_index().rename(columns={
             0: 'user_query_day'
         })
    data = pd.merge(data, user_query_day, 'left', on=['user_id', 'day'])
    user_query_day_hour = data.groupby(
        ['user_id', 'day',
         'hour']).size().reset_index().rename(columns={
             0: 'user_query_day_hour'
         })
    data = pd.merge(
        data, user_query_day_hour, 'left', on=['user_id', 'day', 'hour'])
    user_query_minute = data.groupby(
        ['user_id', 'day', 'hour',
         'minute']).size().reset_index().rename(columns={
             0: 'user_query_minute'
         })
    data = pd.merge(
        data,
        user_query_minute,
        'left',
        on=['user_id', 'day', 'hour', 'minute'])
    return data


df = convert_data(df)

In [4]:
#shop_score = (star + review_num * review_positive) * service * delivery * description
df.shop_star_level = df.shop_star_level - min(df.shop_star_level)

df['shop_score'] = (
    df.shop_star_level +
    df.shop_review_num_level * df.shop_review_positive_rate
) * df.shop_score_service * df.shop_score_delivery * df.shop_score_description

In [6]:
#item_score = (sales + collected + pv)
#item_score2 = (sales + collected + pv) / price
#item_score3 = ((sales + collected + pv) / price) / (page + 1)

df.context_page_id = df.context_page_id - min(df.context_page_id)

df['item_score'] = (
    df.item_sales_level + df.item_collected_level * df.item_pv_level)
df['item_score2'] = (
    df.item_sales_level + df.item_collected_level * df.item_pv_level
) / df.item_price_level
df['item_score3'] = (
    (df.item_sales_level + df.item_collected_level * df.item_pv_level
     ) / df.item_price_level) / (df.context_page_id + 1)

In [7]:
# item与predict相似度
def pred_split(x):
    pred = []
    tmp = x.split(';')
    for i in range(0, len(tmp)):
        pred.append(tmp[i].split(':'))
    return pred


def predict_category_split(x):
    pred = []
    for i in range(0, len(x)):
        pred.append(x[i][0])
    return pred


def predict_property_split(x):
    pred = []
    for i in range(0, len(x)):
        for j in range(1, len(x[i])):
            if (x[i][j] != '-1'):
                pred.append(x[i][j])
    return pred


df['predict_category_property_split'] = df.predict_category_property.apply(
    lambda x: pred_split(x))

df['predict_category_split'] = df.predict_category_property_split.apply(
    lambda x: predict_category_split(x))

df['predict_property_split'] = df.predict_category_property_split.apply(
    lambda x: predict_property_split(x))


def item_pred_category_score(x):
    return len(
        list(
            set(x.item_category_split).intersection(
                set(x.predict_category_split))))


def item_pred_property_score(x):
    return len(
        list(
            set(x.item_property_split).intersection(
                set(x.predict_property_split))))


df['item_category_split'] = df.item_category_list.apply(lambda x: x.split(';'))
df['item_property_split'] = df.item_property_list.apply(lambda x: x.split(';'))
df['predict_category_property_split'] = df.predict_category_property.apply(
    lambda x: x.split(';'))
df['item_category_split_count'] = df.item_category_split.apply(
    lambda x: len(x))
df['item_property_split_count'] = df.item_property_split.apply(
    lambda x: len(x))
df['predict_category_property_split_count'] = df.predict_category_split.apply(
    lambda x: len(x))

df['pred_category_split_count'] = df.predict_category_split.apply(
    lambda x: len(x))
df['pred_property_split_count'] = df.predict_property_split.apply(
    lambda x: len(x))

df['item_pred_category_score'] = df.apply(item_pred_category_score, axis=1)
df['item_pred_property_score'] = df.apply(item_pred_property_score, axis=1)

df['item_pred_category_score_item%'] = df.item_pred_category_score / (
    df.item_category_split_count + 1) * 10
df['item_pred_property_score_item%'] = df.item_pred_property_score / (
    df.item_property_split_count + 1) * 10

df['item_pred_category_score_pred%'] = df.item_pred_category_score / (
    df.pred_category_split_count + 1) * 10
df['item_pred_property_score_pred%'] = df.item_pred_property_score / (
    df.pred_property_split_count + 1) * 10

In [5]:
#用户一个时间段内搜索
df['yesterday'] = df.day - 1
df['yesterhour'] = df.hour.apply(lambda x: x - 1 if x != 0 else 23)

user_day_query = df.groupby(
    ['day', 'user_id'], as_index=False)['user_id'].agg({
        'user_day_query':
        'count'
    })
df = df.merge(user_day_query, on=['day', 'user_id'], how='left')
user_day_query.columns = ['day2', 'user_id', 'user_yesterday_query']
df = df.merge(
    user_day_query,
    left_on=['yesterday', 'user_id'],
    right_on=['day2', 'user_id'],
    how='left')

user_hour_query = df.groupby(
    ['day', 'hour', 'user_id'], as_index=False)['user_id'].agg({
        'user_hour_query':
        'count'
    })
df = df.merge(user_hour_query, on=['day', 'hour', 'user_id'], how='left')
user_hour_query.columns = ['day3', 'hour3', 'user_id', 'user_yesterhour_query']
df = df.merge(
    user_hour_query,
    left_on=['day', 'yesterhour', 'user_id'],
    right_on=['day3', 'hour3', 'user_id'],
    how='left')

user_minute_query = df.groupby(
    ['day', 'hour', 'minute', 'user_id'], as_index=False)['user_id'].agg({
        'user_minute_query':
        'count'
    })
df = df.merge(
    user_minute_query, on=['day', 'hour', 'minute', 'user_id'], how='left')

df = df.drop(['day2', 'day3', 'hour3'], axis=1)
df[[
    'user_day_query', 'user_hour_query', 'user_yesterday_query',
    'user_yesterhour_query', 'user_minute_query'
]] = df[[
    'user_day_query', 'user_hour_query', 'user_yesterday_query',
    'user_yesterhour_query', 'user_minute_query'
]].fillna(0)

ValueError: labels ['day2' 'day3' 'hour3'] not contained in axis

In [7]:
#item一个时间段内被搜索、user_item组合一个时间段内被搜索
item_day_query = df.groupby(
    ['day', 'item_id'], as_index=False)['item_id'].agg({
        'item_day_query':
        'count'
    })
df = df.merge(item_day_query, on=['day', 'item_id'], how='left')

item_hour_query = df.groupby(
    ['day', 'hour', 'item_id'], as_index=False)['item_id'].agg({
        'item_hour_query':
        'count'
    })
df = df.merge(item_hour_query, on=['day', 'hour', 'item_id'], how='left')

item_minute_query = df.groupby(
    ['day', 'hour', 'minute', 'item_id'], as_index=False)['item_id'].agg({
        'item_minute_query':
        'count'
    })
df = df.merge(
    item_minute_query, on=['day', 'hour', 'minute', 'item_id'], how='left')

user_item_day_query = df.groupby(
    ['day', 'item_id', 'user_id'], as_index=False)['user_id'].agg({
        'user_item_day_query':
        'count'
    })
df = df.merge(
    user_item_day_query, on=['day', 'item_id', 'user_id'], how='left')

user_item_hour_query = df.groupby(
    ['day', 'hour', 'item_id', 'user_id'], as_index=False)['user_id'].agg({
        'user_item_hour_query':
        'count'
    })
df = df.merge(
    user_item_hour_query, on=['day', 'hour', 'item_id', 'user_id'], how='left')

user_item_minute_query = df.groupby(
    ['day', 'hour', 'minute', 'item_id', 'user_id'],
    as_index=False)['user_id'].agg({
        'user_item_minute_query': 'count'
    })
df = df.merge(
    user_item_minute_query,
    on=['day', 'hour', 'minute', 'item_id', 'user_id'],
    how='left')

df[[
    'item_day_query', 'item_hour_query', 'item_minute_query',
    'user_item_day_query', 'user_item_hour_query', 'user_item_minute_query'
]] = df[[
    'item_day_query', 'item_hour_query', 'item_minute_query',
    'user_item_day_query', 'user_item_hour_query', 'user_item_minute_query'
]].fillna(0)

In [30]:
#小时变为半小时划分时间段
df.minute = df.minute + 5
df['min_time'] = df.hour * 2 + round(df.minute / 60)

In [3]:
df = pd.read_csv('df_without_his.csv')

In [8]:
df.shape
df.head()
#(496509, 58)

(496509, 68)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_id,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,time,day,hour,user_query_day,user_query_day_hour,shop_score,item_score,item_score2,item_score3,predict_category_property_split,predict_category_split,predict_property_split,item_category_split,item_property_split,item_category_split_count,item_property_split_count,predict_category_property_split_count,pred_category_split_count,pred_property_split_count,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%,yesterday,yesterhour,user_day_query,user_yesterday_query,user_hour_query,user_yesterhour_query,minute,user_query_minute,item_day_query,item_hour_query,user_item_day_query,min_time,user_item_hour_query,user_minute_query,item_minute_query,user_item_minute_query
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,1,1003,2005,3003,282924576738839389,1537236544,5,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,3,1.0,1.0,1.0,0,2018-09-18 10:09:04,18,10,3,3,7.0,59,19.666667,3.277778,"['5799347067982556520:-1', '509660095530134768...","['5799347067982556520', '509660095530134768', ...",['9148482949976129397'],"['7908382889764677758', '5799347067982556520']","['2072967855524022579', '5131280576272319091',...",2,22,5,5,1,2,0,6.666667,0.000000,3.333333,0.000000,17,9,3,0.0,3,0.0,5,1,6,2,1,20.0,1,1,1,1
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,0,1002,2005,3006,4007979028023783431,1537243232,0,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,3,1.0,1.0,1.0,0,2018-09-18 12:00:32,18,12,8,5,7.0,59,19.666667,19.666667,"['5799347067982556520:9172976955054793469', '7...","['5799347067982556520', '7908382889764677758']","['9172976955054793469', '1787573075717641245,9...","['7908382889764677758', '5799347067982556520']","['2072967855524022579', '5131280576272319091',...",2,22,2,2,2,2,1,6.666667,0.434783,6.666667,3.333333,17,11,8,0.0,5,1.0,5,4,6,1,1,24.0,1,4,1,1
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,0,1003,2005,3004,4884875192608989870,1537211052,0,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,3,1.0,1.0,1.0,0,2018-09-18 03:04:12,18,3,2,2,7.0,59,19.666667,19.666667,"['5799347067982556520:5131280576272319091', '7...","['5799347067982556520', '7258015885215914736',...","['5131280576272319091', '5131280576272319091',...","['7908382889764677758', '5799347067982556520']","['2072967855524022579', '5131280576272319091',...",2,22,3,3,3,2,1,6.666667,0.434783,5.000000,2.500000,17,2,2,0.0,2,0.0,5,2,6,2,1,6.0,1,2,1,1
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,2681414445369714628,1,1004,2005,3006,840119421106178602,1537222670,15,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,3,1.0,1.0,1.0,0,2018-09-18 06:17:50,18,6,1,1,7.0,59,19.666667,1.229167,"['509660095530134768:-1', '5799347067982556520...","['509660095530134768', '5799347067982556520', ...","['1787573075717641245,9148482949976129397', '9...","['79083

In [ ]:
#user上次查询、上次交易时间差，item上次查询、上次交易时间差
user_last_query = []
item_last_query = []

for i in range(0, df.shape[0]):
    user_jud = 0
    item_jud = 0
    for j in reversed(range(0, i)):
        if ((df.iloc[j, :].user_id == df.iloc[i, :].user_id) &
            (user_jud == 0)):
            user_last_query.append(df.iloc[j, :].time)
            user_jud = 1
        if ((df.iloc[j, :].item_id == df.iloc[i, :].item_id) &
            (item_jud == 0)):
            item_last_query.append(df.iloc[j, :].time)
            item_jud = 1
    if (user_jud == 0):
        user_last_query.append(-1)
    if (item_jud == 0):
        item_last_query.append(-1)

df['user_last_query'] = user_last_query
df['item_last_query'] = item_last_query

In [ ]:
#user偏好哪类商品

In [ ]:
#user某时间段第几次点击

In [ ]:
#user购买率(occupation/item/brand/city)

In [9]:
df.to_csv('df_without_his.csv', index=False)